In [77]:
from sec_api import QueryApi, ExtractorApi
import json
import requests
import pandas as pd

queryAPI = QueryApi(api_key='8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9')
extractorApi = ExtractorApi(api_key='8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9')

In [78]:
#formType:(“10-K”, “10-KT”, “10KSB”, “10KT405”, “10KSB40”, “10-K405”)
class SEC_QUERY:

    def __init__(self, form_type, ticker, size):
        self.form_type = form_type
        self.ticker = ticker
        self.size = size
        self.query = {
            "query": { 
                "query_string" : {
                    "query": f"formType:\"{self.form_type}\" AND ticker:{self.ticker}",
                }
            },
            "from": "0",
            "size": size,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
    
    def get_response(self, index:int):
        response = queryAPI.get_filings(self.query)
        return json.dumps(response["filings"][index], indent=2)
    

    def extract(self, index:int):
        query_str = self.get_response(index)
        q_dict = json.loads(query_str)
        filing_url = q_dict['linkToHtml']

        xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
        api_key = '8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9'

        final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
        response = requests.get(final_url)

        final_json = json.loads(response.text)
        return final_json, filing_url

In [84]:
query1 = SEC_QUERY("10-K", "AAPL", "10")
#print(query1.get_response(0))
init_query, f_url = query1.extract(0)

# 10-K supported items [1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15]
section_text = extractorApi.get_section(f_url, "1A", "text")

In [85]:
print(section_text)

 Item 1A. Risk Factors 

The Company&#8217;s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Company&#8217;s business, reputation, results of operations, financial condition and stock price can be materially and adversely affected. 

Because of the following factors, as well as other factors affecting the Company&#8217;s results of operations and financial condition, past financial performance should not be considered to be a reliable indicator of future performance, and investors should not use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains forward-looking statements. 

This section should be read in conjunction with Part II, Item 7, &#8220;Management&#8217;s Discussion and Analysis of Financial Condition and Resu

In [69]:
# def extract(query_obj:SEC_QUERY, index: int):
#     #convert query_object string to dict
#     query_str = query_obj.get_response(index)
#     q_dict = json.loads(query_str)
#     filing_url = q_dict['linkToHtml']

#     xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
#     api_key = '8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9'

#     final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
#     response = requests.get(final_url)

#     final_json = json.loads(response.text)
#     return final_json